<a href="https://colab.research.google.com/github/runnin123/Jeju_Bigdata/blob/master/Jeju_bigdata_def.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import bigquery

import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import LabelEncoder

# 평가 메트릭
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 시각화
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt  # Matlab-style plotting

# 통계
from scipy import stats
from scipy.stats import norm, skew #for some statistics

from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from sklearn.model_selection import train_test_split

In [4]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/jeju/201901-202003.csv')

In [5]:
def rmsle(y, pred): 
  log_y = np.log1p(y)
  log_pred = np.log1p(pred)
  squared_error = (log_y - log_pred)**2
  rmsle = np.sqrt(np.mean(squared_error))
  return print('Test Data RMSLE: {0:.3f}'.format(rmsle))

In [6]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

In [7]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
from itertools import product

In [10]:
# 교차검증 수행
def lcl_best_params_model(model, params):
  global cv_model
  
  cv_model = GridSearchCV(model, param_grid=params, scoring="neg_mean_squared_error", cv = 5)
  cv_model.fit(X_train_lcl, y_train_lcl)
  # print("----", model.__class__.__name__, "----")
  # print("GridSearchCV 최적 하이퍼 파라미터 :", cv_model.best_params_)

  rmse = np.sqrt(-1*cv_model.best_score_)
  # print("GridSearchCV 최적 평균 RMSE값 :", np.round(rmse, 3))

  eval_pred = cv_model.predict(X_test_lcl)
  eval_pred = np.expm1(eval_pred)
  rmsle(y_test_lcl, eval_pred)
  
  return cv_model.best_estimator_

In [11]:
def date_pre(train):
  global data

  data = train.copy()
  data = data.fillna('')
  data['REG_YYMM'].head()

  data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
  data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
  data = data.drop(['REG_YYMM'], axis=1)

  return data

In [12]:
def local(data):

  global local
  
  local = data[data['CARD_CCG_NM']==data['HOM_CCG_NM']].reset_index(drop=True)

  # 데이터 정제
  local = local.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)
  columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
  local = local.groupby(columns).sum().reset_index(drop=False)

  return local

In [13]:
def lcl_encoding(local):

  global lcl_num, encoder, encoders

  # 인코딩
  dtypes_lcl = local.dtypes
  encoders = {}
  for column in local.columns:
      if str(dtypes_lcl[column]) == 'object':
          encoder = LabelEncoder()
          encoder.fit(local[column])
          encoders[column] = encoder
          
  lcl_num = local.copy()        
  for column in encoders.keys():
      encoder = encoders[column]
      lcl_num[column] = encoder.transform(local[column])

  return lcl_num

In [14]:
def train_test(lcl_num):
  global X_train_lcl, y_train_lcl, X_test_lcl, y_test_lcl
  
  X_lcl, y_lcl = lcl_num.loc[:, lcl_num.columns != 'AMT'], lcl_num['AMT']
  X_lcl  = X_lcl.drop(['CSTMR_CNT', 'CNT'], axis=1)
  X_train_lcl, X_test_lcl, y_train_lcl, y_test_lcl = train_test_split(X_lcl, y_lcl, test_size=0.3, random_state=126, shuffle=True)
  X_train_lcl.shape, y_train_lcl.shape, X_test_lcl.shape, y_test_lcl.shape
  y_train_lcl = np.log1p(y_train_lcl)

  return X_train_lcl, y_train_lcl, X_test_lcl, y_test_lcl

In [15]:
def pred_model(best_models_lcl):
  global xgb_pred_lcl, gbm_pred_lcl, lgb_pred_lcl, xgb_reg_lcl, gbm_reg_lcl, lgb_reg_lcl

  # lcl 모형 불러오기
  xgb_reg_lcl = best_models_lcl[0]
  gbm_reg_lcl = best_models_lcl[1]
  lgb_reg_lcl = best_models_lcl[2]

  # 예측 데이터 세트
  xgb_pred_lcl = xgb_reg_lcl.predict(X_test_lcl)
  xgb_pred_lcl = np.expm1(xgb_pred_lcl)

  gbm_pred_lcl = gbm_reg_lcl.predict(X_test_lcl)
  gbm_pred_lcl = np.expm1(gbm_pred_lcl)

  lgb_pred_lcl = lgb_reg_lcl.predict(X_test_lcl)
  lgb_pred_lcl = np.expm1(lgb_pred_lcl)

  pred_lcl = np.array([xgb_pred_lcl, gbm_pred_lcl, lgb_pred_lcl])

  # transpose를 이용해 행과 열의 위치 교환
  pred_lcl = np.transpose(pred_lcl)

  final_lcl = xgb_pred_lcl*0.60 + gbm_pred_lcl*0.20 + lgb_pred_lcl*0.20
  rmsle(y_test_lcl, final_lcl)

  return xgb_pred_lcl, gbm_pred_lcl, lgb_pred_lcl, xgb_reg_lcl, gbm_reg_lcl, lgb_reg_lcl

In [16]:
def make_templet(lcl_num):
  global temp_lcl

  # lcl 예측 템플릿 만들기
  CARD_SIDO_NMs = lcl_num['CARD_SIDO_NM'].unique()
  STD_CLSS_NMs  = lcl_num['STD_CLSS_NM'].unique()
  HOM_SIDO_NMs  = lcl_num['HOM_SIDO_NM'].unique()
  AGEs          = lcl_num['AGE'].unique()
  SEX_CTGO_CDs  = lcl_num['SEX_CTGO_CD'].unique()
  FLCs          = lcl_num['FLC'].unique()
  years         = [2020]
  months        = [4, 7]

  comb_list = [CARD_SIDO_NMs, STD_CLSS_NMs,HOM_SIDO_NMs, AGEs, SEX_CTGO_CDs, FLCs, years, months]
  temp = np.array(list(product(*comb_list)))

  train_features = lcl_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
  temp_lcl = pd.DataFrame(data=temp, columns=train_features.columns)

  return temp_lcl

In [17]:
def pred_dataset(temp_lcl):
  global final_lcl
  
  # lcl 예측 데이터 세트
  xgb_pred_lcl = xgb_reg_lcl.predict(temp_lcl)
  xgb_pred_lcl = np.expm1(xgb_pred_lcl)

  gbm_pred_lcl = gbm_reg_lcl.predict(temp_lcl)
  gbm_pred_lcl = np.expm1(gbm_pred_lcl)

  lgb_pred_lcl = lgb_reg_lcl.predict(temp_lcl)
  lgb_pred_lcl = np.expm1(lgb_pred_lcl)

  final_lcl = xgb_pred_lcl*0.30 + gbm_pred_lcl*0.30 + lgb_pred_lcl*0.40

  return final_lcl

In [18]:
def make_sub_lcl(temp_lcl):

  temp_lcl['AMT'] = np.round(final_lcl, 0)
  temp_lcl['REG_YYMM'] = temp_lcl['year']*100 + temp_lcl['month']
  temp_lcl = temp_lcl[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
  temp_lcl = temp_lcl.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

  # 디코딩 
  temp_lcl['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp_lcl['CARD_SIDO_NM'])
  temp_lcl['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp_lcl['STD_CLSS_NM'])

  return temp_lcl

In [ ]:
for i in range(1, 101):
  train = train.sample(n = 50000, random_state = i).sort_values('REG_YYMM').reset_index()
  del train['index']

  print(train['STD_CLSS_NM'][0])

  date_pre(train)
  
  local = data[data['CARD_CCG_NM']==data['HOM_CCG_NM']].reset_index(drop=True)

  # 데이터 정제
  local = local.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)
  columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
  local = local.groupby(columns).sum().reset_index(drop=False)
  
  lcl_encoding(local)

  train_test(lcl_num)

  # 4개의 모델 선언
  # random_state = 0 <- 이 숫자는 아무거나 적어도 상관 X
  # xgb gbm lgb 모두 'random_state=0' <- 모두 같은 데이터로 모델링
  xgb = XGBRegressor(random_state=i)
  gbm = GradientBoostingRegressor(random_state=i)
  lgb = LGBMRegressor(random_state=i)  

  params = {'n_estimators': [1000, 2000]}

  models = [xgb, gbm, lgb] # 모형 하나 추가 함
  best_models_lcl = []
  for model in models:
    new_model = lcl_best_params_model(model=model, params=params)
    best_models_lcl.append(new_model)

  pred_model(best_models_lcl)

  # lcl 예측 템플릿 만들기
  CARD_SIDO_NMs = lcl_num['CARD_SIDO_NM'].unique()
  STD_CLSS_NMs  = lcl_num['STD_CLSS_NM'].unique()
  HOM_SIDO_NMs  = lcl_num['HOM_SIDO_NM'].unique()
  AGEs          = lcl_num['AGE'].unique()
  SEX_CTGO_CDs  = lcl_num['SEX_CTGO_CD'].unique()
  FLCs          = lcl_num['FLC'].unique()
  years         = [2020]
  months        = [4, 7]

  comb_list = [CARD_SIDO_NMs, STD_CLSS_NMs,HOM_SIDO_NMs, AGEs, SEX_CTGO_CDs, FLCs, years, months]
  temp = np.array(list(product(*comb_list)))

  train_features = lcl_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
  temp_lcl = pd.DataFrame(data=temp, columns=train_features.columns)

  # lcl 예측 데이터 세트
  xgb_pred_lcl = xgb_reg_lcl.predict(temp_lcl)
  xgb_pred_lcl = np.expm1(xgb_pred_lcl)

  gbm_pred_lcl = gbm_reg_lcl.predict(temp_lcl)
  gbm_pred_lcl = np.expm1(gbm_pred_lcl)

  lgb_pred_lcl = lgb_reg_lcl.predict(temp_lcl)
  lgb_pred_lcl = np.expm1(lgb_pred_lcl)

  final_lcl = xgb_pred_lcl*0.30 + gbm_pred_lcl*0.30 + lgb_pred_lcl*0.40

  temp_lcl['AMT'] = np.round(final_lcl, 0)
  temp_lcl['REG_YYMM'] = temp_lcl['year']*100 + temp_lcl['month']
  temp_lcl = temp_lcl[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
  temp_lcl = temp_lcl.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

  # 디코딩 
  temp_lcl['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp_lcl['CARD_SIDO_NM'])
  temp_lcl['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp_lcl['STD_CLSS_NM'])

  globals()['temp_lcl_{}'.format(i)] = temp_lcl

일식 음식점업
[01:52:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:52:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:52:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:52:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:52:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:52:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:52:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:52:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in fav

In [ ]:
# for i in range(1, 4):
#   train = train.sample(n = 5000, random_state = i).sort_values('REG_YYMM').reset_index()
#   del train['index']

#   print(train['STD_CLSS_NM'][0])

#   date_pre(train)

#   local = data[data['CARD_CCG_NM']==data['HOM_CCG_NM']].reset_index(drop=True)
#   tourist = data[data['CARD_CCG_NM']!=data['HOM_CCG_NM']].reset_index(drop=True)

#   # 데이터 정제
#   local = local.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)
#   columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
#   local = local.groupby(columns).sum().reset_index(drop=False)

#     # 인코딩
#   dtypes_lcl = local.dtypes
#   encoders = {}
#   for column in local.columns:
#       if str(dtypes_lcl[column]) == 'object':
#           encoder = LabelEncoder()
#           encoder.fit(local[column])
#           encoders[column] = encoder
          
#   lcl_num = local.copy()        
#   for column in encoders.keys():
#       encoder = encoders[column]
#       lcl_num[column] = encoder.transform(local[column])

#   X_lcl, y_lcl = lcl_num.loc[:, lcl_num.columns != 'AMT'], lcl_num['AMT']
#   X_lcl  = X_lcl.drop(['CSTMR_CNT', 'CNT'], axis=1)
#   X_train_lcl, X_test_lcl, y_train_lcl, y_test_lcl = train_test_split(X_lcl, y_lcl, test_size=0.3, random_state=126, shuffle=True)
#   X_train_lcl.shape, y_train_lcl.shape, X_test_lcl.shape, y_test_lcl.shape

#   y_train_lcl = np.log1p(y_train_lcl)

#   # 4개의 모델 선언
#   # random_state = 0 <- 이 숫자는 아무거나 적어도 상관 X
#   # xgb gbm lgb 모두 'random_state=0' <- 모두 같은 데이터로 모델링
#   xgb = XGBRegressor(random_state=i)
#   gbm = GradientBoostingRegressor(random_state=i)
#   lgb = LGBMRegressor(random_state=i)  

#   params = {'n_estimators': [1000, 2000]}

#   models = [xgb, gbm, lgb] # 모형 하나 추가 함
#   best_models_lcl = []
#   for model in models:
#     new_model = lcl_best_params_model(model=model, params=params)
#     best_models_lcl.append(new_model)

#   # lcl 모형 불러오기
#   xgb_reg_lcl = best_models_lcl[0]
#   gbm_reg_lcl = best_models_lcl[1]
#   lgb_reg_lcl = best_models_lcl[2]

#   # 예측 데이터 세트
#   xgb_pred_lcl = xgb_reg_lcl.predict(X_test_lcl)
#   xgb_pred_lcl = np.expm1(xgb_pred_lcl)

#   gbm_pred_lcl = gbm_reg_lcl.predict(X_test_lcl)
#   gbm_pred_lcl = np.expm1(gbm_pred_lcl)

#   lgb_pred_lcl = lgb_reg_lcl.predict(X_test_lcl)
#   lgb_pred_lcl = np.expm1(lgb_pred_lcl)

#   pred_lcl = np.array([xgb_pred_lcl, gbm_pred_lcl, lgb_pred_lcl])

#   # transpose를 이용해 행과 열의 위치 교환
#   pred_lcl = np.transpose(pred_lcl)

#   final_lcl = xgb_pred_lcl*0.60 + gbm_pred_lcl*0.20 + lgb_pred_lcl*0.20
#   rmsle(y_test_lcl, final_lcl)

#   # lcl 예측 템플릿 만들기
#   CARD_SIDO_NMs = lcl_num['CARD_SIDO_NM'].unique()
#   STD_CLSS_NMs  = lcl_num['STD_CLSS_NM'].unique()
#   HOM_SIDO_NMs  = lcl_num['HOM_SIDO_NM'].unique()
#   AGEs          = lcl_num['AGE'].unique()
#   SEX_CTGO_CDs  = lcl_num['SEX_CTGO_CD'].unique()
#   FLCs          = lcl_num['FLC'].unique()
#   years         = [2020]
#   months        = [4, 7]

#   comb_list = [CARD_SIDO_NMs, STD_CLSS_NMs,HOM_SIDO_NMs, AGEs, SEX_CTGO_CDs, FLCs, years, months]
#   temp = np.array(list(product(*comb_list)))

#   train_features = lcl_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
#   temp_lcl = pd.DataFrame(data=temp, columns=train_features.columns)

#   # lcl 예측 데이터 세트
#   xgb_pred_lcl = xgb_reg_lcl.predict(temp_lcl)
#   xgb_pred_lcl = np.expm1(xgb_pred_lcl)

#   gbm_pred_lcl = gbm_reg_lcl.predict(temp_lcl)
#   gbm_pred_lcl = np.expm1(gbm_pred_lcl)

#   lgb_pred_lcl = lgb_reg_lcl.predict(temp_lcl)
#   lgb_pred_lcl = np.expm1(lgb_pred_lcl)

#   final_lcl = xgb_pred_lcl*0.60 + gbm_pred_lcl*0.20 + lgb_pred_lcl*0.20

#   temp_lcl['AMT'] = np.round(final_lcl, 0)
#   temp_lcl['REG_YYMM'] = temp_lcl['year']*100 + temp_lcl['month']
#   temp_lcl = temp_lcl[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
#   temp_lcl = temp_lcl.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

#   # 디코딩 
#   temp_lcl['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp_lcl['CARD_SIDO_NM'])
#   temp_lcl['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp_lcl['STD_CLSS_NM'])

#   globals()['temp_lcl_{}'.format(i)] = temp_lcl